In [2]:
import pandas as pd
df=pd.read_csv('/Users/AFROARC/Documents/GitHub/IronHackLabs/Week 6/storytelling/movement-range-data-2021-11-21/movement-range-2021-11-21.txt', sep ='\t')
df.head()

/opt/anaconda3/envs/classenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ds,country,polygon_source,polygon_id,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,baseline_name,baseline_type
0,2021-01-01,AGO,GADM,AGO.10.10_1,Lubango,-0.35291,0.25398,full_february,DAY_OF_WEEK
1,2021-01-02,AGO,GADM,AGO.10.10_1,Lubango,-0.06131,0.17330,full_february,DAY_OF_WEEK
2,2021-01-03,AGO,GADM,AGO.10.10_1,Lubango,-0.00392,0.21932,full_february,DAY_OF_WEEK
3,2021-01-04,AGO,GADM,AGO.10.10_1,Lubango,0.15114,0.11662,full_february,DAY_OF_WEEK
4,2021-01-05,AGO,GADM,AGO.10.10_1,Lubango,0.12696,0.10832,full_february,DAY_OF_WEEK


In [ ]:
df.head()

In [3]:
continents <- vroom("../data/animation/continents.csv")


# https://www.apple.com/covid19/mobility
movement_data <- vroom("https://github.com/ActiveConclusion/COVID19_mobility/raw/master/apple_reports/applemobilitytrends.csv") %>%
  filter(geo_type != "city") %>% select(-geo_type) %>%
  gather(key = "date", value = "mobility", -region,-transportation_type) %>%
  rename(country=region) %>%
  mutate(date=ymd(date)) %>%
  mutate(mobility = as.numeric(mobility))

# http://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv
covid <- vroom("http://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv") %>% 
  select(-continent) %>%
  rename(country=location) %>%
  arrange(country,date) %>%
  group_by(country) %>%
  fill(total_cases_per_million, .direction = "downup") %>%
  right_join(continents,by="country") %>%
  rename(population = population.y)



combined <- movement_data %>% 
  full_join(covid, by=c("country","date")) %>%
  filter(!is.na(transportation_type)) %>%
  filter(date>("2020-02-25")) %>%
  group_by(country,transportation_type) %>%
  fill(total_cases_per_million, .direction = "downup") %>%
  fill(continent, .direction = "downup") %>%
  fill(population, .direction = "downup")



fig <-   combined %>% 
  filter(transportation_type=="driving") %>%
  mutate(date = as.character(date)) %>%
  arrange(country, transportation_type, date) %>%
  mutate(total_cases_per_million = total_cases_per_million+2) %>%
  plot_ly(
    x = ~total_cases_per_million,
    y = ~mobility,
    size = ~log(population),
    color = ~continent,
    frame = ~date,
    ids = ~country,
    text = ~country,
    hoverinfo = "text",
    type = 'scatter',
    mode = 'markers'
  )
fig <- fig %>% layout(
  xaxis = list(
    type = "log",
    title = "Covid-19 cases per million"
  ), 
  yaxis = list(
    title = "Mobility index (driving)"
  )
) %>% config(displayModeBar = FALSE)

fig

SyntaxError: invalid syntax (3707385136.py, line 5)